In [20]:
import pandas as pd


In [21]:
data_url= "http://archive.ics.uci.edu/ml/machine-learning-databases/postoperative-patient-data/post-operative.data"
names = ['L-CORE', 'L-SURF','L-02','L-BP','SURF-STBL','CORE-STBL','BP-STBL','COMFORT','decision']

df = pd.read_csv(data_url,names=names)
df =df.drop([3])
df[:5]

,L-CORE,L-SURF,L-02,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
4,mid,mid,excellent,high,stable,stable,stable,10,A
5,high,low,good,mid,stable,stable,unstable,15,S


In [22]:
df.shape

(89, 9)

In [23]:
df.dtypes

L-CORE       object
L-SURF       object
L-02         object
L-BP         object
SURF-STBL    object
CORE-STBL    object
BP-STBL      object
COMFORT      object
decision     object
dtype: object

In [24]:
for col in names:
    df[col] = df[col].astype('category')
df.dtypes    

L-CORE       category
L-SURF       category
L-02         category
L-BP         category
SURF-STBL    category
CORE-STBL    category
BP-STBL      category
COMFORT      category
decision     category
dtype: object

In [25]:

df_modified = pd.get_dummies(df)
df_modified[:5]

,L-CORE_high,L-CORE_low,L-CORE_mid,L-SURF_high,L-SURF_low,L-SURF_mid,L-02_excellent,L-02_good,L-BP_high,L-BP_low,...,BP-STBL_stable,BP-STBL_unstable,COMFORT_05,COMFORT_07,COMFORT_10,COMFORT_15,COMFORT_?,decision_A,decision_I,decision_S
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [26]:
x_data = df_modified.ix[:, :24].values
y_data = df_modified.ix[:,-3: ].values

#print(y_data)

In [ ]:
import numpy as np 

training_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.8))
test_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.2))

x_training, x_test = x_data[training_idx,:], x_data[test_idx,:]
y_training, y_test = y_data[training_idx,:], y_data[test_idx,:]

#print(x_training)
#x_training.shape[1]
#print(y_training)
y_training.shape[1]

3

In [ ]:
import tensorflow as tf

X = tf.placeholder("float", [None, len(x_data[0])])
Y = tf.placeholder("float", [None, len(y_data[0])])

W = tf.Variable(tf.zeros([len(x_data[0]), len(y_data[0])]))

hypothesis = tf.nn.softmax(tf.matmul(X,W))
learning_rate = 0.001

cost = tf.reduce_mean(-1 * tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

for num in range(5):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    init = tf.initialize_all_variables()

    print("-------------",num+1,"번째------------")
    
    sess = tf.Session()
    sess.run(init)
    
    per_result = []
    
    for i in range(20000000):
        sess.run(optimizer, feed_dict={X: x_training, Y:y_training})
        if i % 2000000 == 0:
            print ("%d 's iteration" % i)
            print (sess.run(cost, feed_dict={X: x_training, Y:y_training}))
    finale_theta = sess.run(W)
    
    hypotehsis_value = sess.run(hypothesis, feed_dict={X:x_test})
    result= [np.argmax(predict) == np.argmax(original_value)  for predict, original_value in zip(hypotehsis_value, y_test)]
    per = sum(result) / len(result)
    per_result.append(per)
    print(per)

------------- 1 번째------------
0 's iteration
1.09766
